<a href="https://colab.research.google.com/github/RexGod/DataMining_Pgu_Nueral_Network/blob/main/finalDeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import tensorflow as tf

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2

In [35]:
pathImages = '/content/drive/MyDrive/tits_pic'

In [36]:
pathManual = '/content/drive/MyDrive/Radiology_manual_annotations.xlsx'
manualRadio = pd.read_excel(pathManual)

In [37]:
manualRadio= manualRadio.rename(columns={'Pathology Classification/ Follow up':'classification'})

In [38]:
manualRadio = manualRadio[manualRadio['Type']=='CESM']

In [39]:
import os
images = os.listdir(pathImages)
loaded_images = []
#filename_txt = []
for filename in images:
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_list = plt.imread(os.path.join(pathImages, filename))
        loaded_images.append(image_list)
        #filename_txt.append(filename)

In [40]:
manualRadio = manualRadio.sort_values(by=['Image_name'])

In [41]:
validation_data_size = int(len(loaded_images)*0.12)
test_data_size = int(len(loaded_images)*0.13)

In [42]:
random_select = np.random.choice(len(loaded_images), size=validation_data_size, replace=False)

In [43]:
validation_data_x = np.array(loaded_images)[random_select]
validation_data_y = manualRadio.iloc[random_select]
loaded_images = np.delete(loaded_images, random_select, axis=0)
manualRadio=manualRadio.drop(manualRadio.index[random_select])

In [44]:
random_select = np.random.choice(len(loaded_images), size=test_data_size, replace=False)

In [45]:
test_data_x = np.array(loaded_images)[random_select]
test_data_y = manualRadio.iloc[random_select]
loaded_images = np.delete(loaded_images, random_select, axis=0)
manualRadio=manualRadio.drop(manualRadio.index[random_select])

In [46]:
train_data_x = np.array(loaded_images)
train_data_y = manualRadio[["Image_name" , "classification"]]

In [47]:
import tensorflow as tf
import keras

In [48]:
label_mapping = {'Benign': 0, 'Normal': 1, 'Malignant': 2}

In [49]:
integer_labels = train_data_y['classification'].map(label_mapping)
integer_labels_valid = validation_data_y['classification'].map(label_mapping)
y_train_to_categorical = keras.utils.to_categorical(integer_labels, num_classes=len(label_mapping), dtype='float32')
y_validation_to_categorical = keras.utils.to_categorical(integer_labels_valid, num_classes=len(label_mapping), dtype='float32')

In [50]:
from tensorflow.keras.applications.resnet import ResNet101,ResNet152 , preprocess_input , decode_predictions

In [51]:
#train_images_rgb = np.repeat(train_data_x[..., np.newaxis], 3, -1)[..., 0]
#test_images_rgb = np.repeat(test_data_x[..., np.newaxis], 3, -1)
#validation_images_rgb = np.repeat(validation_data_x[..., np.newaxis], 3, -1)[..., 0]


In [52]:
train_data_x = train_data_x/255
#preprocess_images_test = preprocess_input(test_images_rgb)
validation_data_x = validation_data_x/255

In [53]:
base_model = ResNet152(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [54]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D , Conv2D , Flatten , Dropout , MaxPooling2D , BatchNormalization
from keras import layers
from tensorflow.keras.optimizers import Adam,Adamax,SGD


In [55]:
model = Sequential()

In [56]:
model.add(base_model)

In [57]:
for layer in base_model.layers[:150]:
    layer.trainable = False
for layer in base_model.layers[150:]:
    layer.trainable = True

In [58]:
model.add(Flatten())
model.add(Dense(512 , activation = 'LeakyReLU'))
model.add(layers.Dropout(0.3))
model.add(Dense(256 , activation='LeakyReLU'))
model.add(layers.Dropout(0.2))
model.add(Dense(64 , activation='LeakyReLU'))
model.add(layers.Dropout(0.4))
model.add(Dense(32 , activation='relu'))
model.add(layers.Dropout(0.2))
model.add(Dense(3,activation = 'softmax'))

In [59]:
model.compile(optimizer=SGD(learning_rate = 0.0001 , momentum=0.05), loss='categorical_crossentropy', metrics=['accuracy'])

In [60]:
from tensorflow.keras.callbacks import EarlyStopping

In [61]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

In [62]:
nemudar = model.fit(train_data_x, y_train_to_categorical,
                    validation_data=(validation_data_x,y_validation_to_categorical ),
                    epochs=30,
                    callbacks = [early_stopping]
                    )


Epoch 1/30
24/24 [==============================] - 62s 881ms/step - loss: 1.5879 - accuracy: 0.3679 - val_loss: 1.0857 - val_accuracy: 0.4500
Epoch 2/30
24/24 [==============================] - 16s 657ms/step - loss: 1.4037 - accuracy: 0.3373 - val_loss: 1.0838 - val_accuracy: 0.4500
Epoch 3/30
24/24 [==============================] - 15s 615ms/step - loss: 1.3336 - accuracy: 0.3785 - val_loss: 1.0936 - val_accuracy: 0.4500
Epoch 4/30
24/24 [==============================] - 15s 625ms/step - loss: 1.3103 - accuracy: 0.3214 - val_loss: 1.0878 - val_accuracy: 0.4333
Epoch 5/30
24/24 [==============================] - 16s 658ms/step - loss: 1.2286 - accuracy: 0.3838 - val_loss: 1.0829 - val_accuracy: 0.3667
Epoch 6/30
24/24 [==============================] - ETA: 0s - loss: 1.1948 - accuracy: 0.3652

KeyboardInterrupt: ignored